<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>



In [0]:
random_seed = 42

import torch
import random
import numpy as np

random.seed(random_seed)

np.random.seed(random_seed)
torch.manual_seed(random_seed)
# Включите куду
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


## Погружение в Natural Language Proccessing

Так как обработкой языка занимались давно, а нейронные сети пытаются вставить везде лишь лет 10, то прежде чем решать задачи с помощью NN хотелось бы попробовать решить ее классическими методами. 


В данном задании мы будем всеми силами пытаться построить языковую модель (language model). 

Что же такое языковая модель? (если вдруг кто-то забыл после лекции)

Сейчас прольем немного математики, а ниже опишем обычными словами все это.

Допустим, у нас есть огромный текстовый корпус содержащий какой-либо смысл и нам хотелось бы уметь создавать такой текст с помощью обученной модели. Для этого нам необходимо знать вероятность $p(W) = p(w_1, w_2, w_3,..) $ появления предложения или последовательности слов или же нам интересно получить вероятность $p(w_5| w_1, w_2, w_3, w_4)$ появления слова при контексте. Будем называть модели, которые считают такие вероятности языковыми моделями.

#### Что это за модель? 

Тут везде почему-то говорится про какую-то неведомую модель. Уточню, вообще говоря, модель (в статистическом смысле, конечно) является математическим представлением процесса. Почти всегда модели являются приближением процесса. Для этого есть несколько причин, но два наиболее важных:
1. Обычно мы наблюдаем только процесс ограниченное количество раз
2. Модель может быть исключительно сложной, поэтому мы ее упрощаем

Вот что обычно делает модель: она описывает, как моделируемый процесс __создает данные__. В нашем случае моделируемым явлением является человеческий язык. Языковая модель дает нам способ генерации человеческого языка. Эти модели обычно составлены из вероятностных распределений.

Модель построена путем наблюдения некоторых образцов, сгенерированных явлением, которое должно быть смоделировано. Таким же образом, языковая модель строится путем наблюдения за некоторым текстом.



### Bag Of Words (мешок слов) 

Так как мы решили немного проникнуться nlp, стоит обязательно познать абстракцию мешка слов.  Вообще эта идея, есть наиболее простой способ сопоставления слова-вектор( ведь компьютер у нас может работать только с цифрами, поэтому надо каким-то образом уметь превращать буквы/слова в них) Это, безусловно, самый простой способ моделирования человеческого языка. Это не значит, что это бесполезно и непопулярно.

Идея очень проста: посчитаем частоту встречания слов в словаре и составим из этого один вектор.

![img](https://hsto.org/getpro/habr/post_images/b91/46a/5be/b9146a5be315f422479e40d85e995289.jpg)

Попробуем закодить для настоящего корпуса собраний сочинений Федора Михайловича Достоевского

In [0]:
! wget https://raw.githubusercontent.com/DLSchool/dlschool_old/master/materials/homeworks/hw09/dostoevsky.txt

In [0]:
import io

with io.open('dostoevsky.txt', 'r',encoding='utf8') as f:
    text = f.read().replace(u'\xa0', u' ').replace(u'\ufeff','')

In [0]:
import nltk
from nltk import word_tokenize
nltk.download('punkt')
words = word_tokenize(text)

In [0]:
len(words), len(sents)

Вопрос 1: Сколько слов и предложений в датасете?

hint: используйте  word_tokenize & sent_tokenize из nltk

In [0]:
from nltk import sent_tokenize

sents_full = sent_tokenize(text)
sents = [word_tokenize(s) for s in sents_full]

In [0]:
from nltk.corpus import reuters
from collections import Counter
 
counts = Counter(words)
total_count = len(words)

# Напечатайте 20 наиболее популярных слов hint: use counter methods
 
# Посчитайте их частоту встречания hint: you know total count

# Внимание: частота в данном контексте является вероятностью появления слова (независимой)
# Поэтому проверьте основное свойство - суммирование  в 1


**Вопрос 2**

Каковы частоты для слов 
"бесы", '"семья", "брат"'. Введите значение округленное до 5 знаков с помощью функции round

In [0]:
import random
 
# Код для генерации текста уже имеющейся моделью
new_text = []
 
for _ in range(30):
    accumulator = .0
    r = random.random()
    for word, freq in counts.items():
        accumulator += freq
 
        if accumulator >= r:
            new_text.append(word)
            break

print(' '.join(new_text))

Еще один простой вопрос.

**Вопрос №3.**
Попрубйте сгенерировать текст, который состои из слов с вероятностями . появления от 0. 85 до 0.95  и итоговое предложение состоит из 10 слов.

In [45]:

print(' '.join(new_text))

он… замирающим веков правильным всматриваться черепа иеромонахи солжем откупорил победе


Как вы можете видеть, это не самый выразительный фрагмент контента. Полученный текст следует только частотным правилам языка и не более того.


#### Биграммы,  триграммы и n-граммы

Одна идея, которая может помочь нам создать более качественный текст, состоит в том, чтобы убедиться, что новое слово, которое мы добавляем в последовательность, хорошо сочетается с уже существующими словами. Проверка правильности слова после 10 слов может немного переборщить. Мы можем упростить все, чтобы проблема была разумной. Давайте сделаем так, чтобы новое слово получилось после последнего слова в последовательности (модель биграмм) или двух последних слов (модель триграмм) ( можно так долго продолжат, из эмпирических наблюдений при n=5 модель всегда выдает нормальный текст ) .

«Биграм» - это причудливое имя для двух последовательных слов, в то время как триграмма (как вы уже догадались) триплет последовательных слов. Вот некоторые быстрые магии NLTK (библиотека для nlp) для извлечения биграмм / триграмм:

In [0]:
from nltk import bigrams, trigrams
from collections import Counter, defaultdict
 
first_sentence = word_tokenize(sents_full[0])
print(first_sentence)

# Get the bigrams
print(list(bigrams(first_sentence)))

# Get the padded bigrams
print(list(bigrams(first_sentence, pad_left=True, pad_right=True)))
 
# Get the trigrams
print(list(trigrams(first_sentence)))
 
# Get the padded trigrams
print(list(trigrams(first_sentence, pad_left=True, pad_right=True)))

Теперь можно и модель над этими биграммами построить. 

In [0]:
model = defaultdict(lambda: defaultdict(lambda: 0))
 
for sentence in sents:
    for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
        model[(w1, w2)][w3] += 1
        
# следует теперь привести количество встречаемостей к частотам
for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    for - in -:
        приводим

**Вопрос 4**
Какова вероятность слова $w_3$ при условии $w_1, w_2$
выпишите три числа для $w_3 \in [\text{вам, слона, коридорчику}]$

А соответсвующие им биграммы: {[чтоб еще], [мухи, сочинили], [прямо, по]}

Не перепутайтн порядок.

Можете поискать еще интересных сочитаний в модели. Например найти самое часто встречаемое в тексте выражение. Или попробовать использовать пятиграммную модель. Это довольно интересно. 

Ниже, наконец представлен код для нашей генеративной модели. Рекомендую с ним так же поиграться.

In [0]:
new_text = [None, None]
 
sentence_finished = False
 
while not sentence_finished:
    r = random.random()
    accumulator = .0
    for word in model[tuple(new_text[-2:])].keys():
        accumulator += model[tuple(new_text[-2:])][word]
        if accumulator >= r:
            new_text.append(word)
            break
 
    if new_text[-2:] == [None, None]:
        sentence_finished = True

print(' '.join([t for t in new_text if t]))

In [0]:
start = 'Я тогда это говорил'

new_text = word_tokenize(start)
 
sentence_finished = False
 
while not sentence_finished:
    r = random.random()
    accumulator = .0
    for word in model[tuple(new_text[-2:])].keys():
        accumulator += model[tuple(new_text[-2:])][word]
        if accumulator >= r:
            new_text.append(word)
            break
 
    if new_text[-2:] == [None, None]:
        sentence_finished = True

print(' '.join([t for t in new_text if t]))


**Вопрос 5**

Сгенерируйте предложение в ячейке выше 10 раз и введите символ, который наиболее часто появлялся в вашем итоговом предложении.

ans_characters

На данном этапе моедль уже может генерировать более осмысленный текст. Кроме того существует множество статистических методом, которые могают сгладить распределение слов, бороться с появление новых n-грамм. И такие модели до сих пор могут спокойно соревноваться с нейронными решениями. 

Собственно покатили в нейронный мир.

## char-RNN in PyTorch

Тут нам предстоит построить character-level RNN c помощью PyTroch. Для того чтобы освежить свои знания о данной модели рекомендую к прочтению [сию статью](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) написанную Андреем Карпатым.  Сразу проясню, тут мы будем 

In [0]:
import codecs
import io

from torch import nn
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Обновите текст, если что-то потерлось. Можно сделать это не только в памяти компьютера, но и в своей собственной.

In [0]:
with io.open('dostoevsky.txt', 'r',encoding='utf8') as f:
    text = f.read().replace(u'\xa0', u' ').replace(u'\ufeff','')

In [0]:
text[:300]

Закодируем наш текст в цифры, как мы и обсуждали ранее.

In [0]:
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}
encoded = np.array([char2int[ch] for ch in text])

In [0]:
encoded.shape # Наш словарь получился очень большой

#### Обработка данных

Будем использовать для представления букв one-hot вектора. Напишите функцию для этого.

In [0]:
def one_hot_encode(arr, n_labels):
    
    # Инициализируем вектора 
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # заполним 1 в соответсвующем месте
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Приводим к нужному размеру
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

Попробуйте сделать функцию ниже, которая строит генератор мини-батчей. Каждая последовательность будет длины `n_steps`

In [0]:
def get_batches(arr, n_seqs, n_steps):
    """
    Создание генератора, возвращающего минибатчи размера (n_seqs x seq_len) Numpy
    """    
    batch_size = n_seqs * n_steps
    n_batches = len(arr)//batch_size
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size]
    # Reshape into n_seqs rows
    arr = arr.reshape((n_seqs, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n:n+n_steps]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+n_steps]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

### Построение charRNN модели

Ниже вам будет необходимо написать свою char-rnn по данному описанию. Как всегда основные рекомендации: сначала пишем определение слоев в  init, затем описываем их вызов в forward.


Кроме того тут есть важная функция predict. (в блоге Карпатого очень подробно все описано и дабы не заниматься копипастой, я направляю вас туда). Для начала попробуйте проверить учится ли ваша сеть и только потом заполняйте метод predict.

Почему даем именно модель над символами? Потому что это очень простая структура, которая может быть применена не только к текстам, но и к музыке. 

In [0]:
class CharRNN(nn.Module):
    def __init__(self, tokens, n_steps=100, n_hidden=256, n_layers=2,
                               drop_prob=0.1, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        self.dropout = <dropout>
        self.lstm = <torch gru will be the best choice>
        self.fc = <linear>
        
        self.init_weights()
        
    def forward(self, x, hc):
        ''' Forward pass through the network '''
        x, h = self.lstm(x, hc)
#         x = self.dropout(x)
        x = self.fc(x)
        return x, h
    
    def predict(self, char, h=None, top_k=None):
        """        
            Returns the predicted character and the hidden state.
        """
        
        if h is None:
            h = self.init_hidden(1).to(device)
        
        x = np.array([[self.char2int[char]]])
        x = <one-hot>
        
        inputs = torch.from_numpy(x).to(device)
        out, h = self.forward(inputs, h)

        p = <Get proba vector (softmax on last dimension)>
        
        if top_k is None:
            top_ch = np.arange(len(self.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        p = p.numpy().squeeze()
        # Choose 1/k 
        char = np.random.choice(top_ch, p=p/p.sum())
        
        return self.int2char[char], h
    
    def init_weights(self):
        ''' Initialize weights for fully connected layer '''        
        # Set bias tensor to all zeros
        self.fc.bias.data.fill_(0)
        # FC weights as random uniform
        self.fc.weight.data.uniform_(-1, 1)
        
    def init_hidden(self, n_seqs):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x n_seqs x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, n_seqs, self.n_hidden).zero_()
        

Для проверки функционирования у нас есть функция `train` , которая позволит провести вам большое число экспериментов.

In [0]:
def train(net, data, epochs=10, n_seqs=10, n_steps=50, lr=0.0005, clip=3, val_frac=0.1, cuda=False, print_every=200):
    net.train()
#     opt = torch.optim.Adam(net.parameters(), lr=lr)
    opt = <Optimizer : Adam is the best choice>
    criterion = <CE> 
    
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        h = net.init_hidden(n_seqs)
        for x, y in get_batches(data, n_seqs, n_steps):
            counter += 1
            
            # Кодируем данные и отправлячем
            x = one_hot_encode(x, n_chars)
            inputs = torch.from_numpy(x).to(device)
            targets = torch.from_numpy(y).to(device)
            
            #  замените на .copy будет работать стабильнее
            h = torch.tensor(h.data, device=device)

            net.zero_grad()
            
            output, h = net.forward(inputs, h)
            
            loss = criterion(output.view(n_seqs*n_steps,-1), targets.view(n_seqs*n_steps))
            loss.backward()
            
            # clip grad norm может вам помочь
            nn.utils.clip_grad_norm_(<TODO>)

            opt.step()
            
            if counter % print_every == 0:
                net.eval()
                # Get validation loss
                val_h = net.init_hidden(n_seqs)
                val_losses = []
                for x, y in get_batches(val_data, n_seqs, n_steps):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    inputs = torch.from_numpy(x).to(device)
                    targets = torch.from_numpy(y).to(device)
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = val_h.clone().detach()
                    output, val_h = net.forward(inputs, val_h)
                    val_loss = criterion(output.view(n_seqs*n_steps,-1), targets.view(n_seqs*n_steps))
                    val_losses.append(val_loss.item())
                   
                # Попробуем валидироваться таким способом
                prime = 'Дом'
                top_k = 2
                chars = [ch for ch in prime]
                vh = None
                for ch in prime:
                    char, vh = net.predict(ch, vh, top_k=top_k)
                for ii in range(10):
                    char, vh = net.predict(chars[-1], vh, top_k=top_k)
                    chars.append(char)
                    
                chars.append(char)
                print(' '.join(chars))
                net.train()
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)),
                       "Validation Perplexity: {:.4f}".format(np.exp(np.mean(val_losses))))
    return net

## Время тренировки!

 
Теперь мы можем тренировать сеть. Сначала мы создадим саму сеть, с некоторыми заданными гиперпараметрами. Затем определите размеры мини-партий (количество последовательностей и количество шагов) и начните обучение. С функцией поезда мы можем установить количество эпох, скорость обучения и другие параметры. Кроме того, мы можем запустить обучение на графическом процессоре, установив `cuda = True`. Сейчас google дает всем бесплатно использовать gpu с помощью сервиса codelab.


![a](https://www.apmpodcasts.org/wp-content/uploads/2015/06/adventure-time.jpg)

In [0]:
if 'net' in locals():
    del net

In [0]:
num_hidden_units = <подберите оптимальное значение (помните, что это буквы)>
net = CharRNN(chars, n_hidden=num_hidden_units, n_layers=2).to(device)

In [0]:
n_seqs, n_steps = 128, 100
print_every = 100
train(net, encoded, epochs=15, n_seqs=n_seqs, n_steps=n_steps, lr=0.0005, cuda=True, print_every=print_every)

![train](https://i.pinimg.com/474x/0e/58/69/0e5869297852211e8447d6b09fa1f4f5.jpg)

**Вопрос 6**

Введите целую часть итоговой  перплексии ex. [142.37 ] = 142

### Загрузка модели 

Чтобы настроить гиперпараметры для получения максимальной производительности, вам понадобятся наблюдения за обучением и валидацией. Если ваша потеря обучения намного ниже, чем потеря проверки, вы перерабатываете. Увеличьте регуляризацию (больше выпадений) или используйте меньшую сеть. Если потери обучения и проверки близки, вы недофинансируете, чтобы увеличить размер сети.

После обучения мы сохраним модель, чтобы мы могли загрузить ее позже, если понадобится. Здесь сохраняются параметры, необходимые для создания той же архитектуры, гиперпараметров скрытого слоя и текстовых символов.

In [0]:
checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}
with open('./rnn.net', 'wb') as f:
    torch.save(checkpoint, f)

## Семплирование

Теперь, когда модель обучена, мы захотим попробовать ее. Чтобы заполучить текст, мы передаем символ и сеть прогнозируем следующий символ. Затем мы берем новый символ, передаем его обратно и получаем еще один. Просто продолжайте делать это, и вы создадите кучу текста!

### Top K 

Наши прогнозы основаны на категориальном распределении вероятностей по всем возможным признакам. Мы можем сделать выборку более разумным, но менее переменным, учитывая только некоторые вероятные символы $ K $. Это будет препятствовать тому, чтобы сеть давала нам абсолютно абсурдные символы, позволяя ей вводить некоторый шум и случайность в выбранный текст.

Как правило, вы хотите настроить сеть, чтобы создать скрытое состояние. В противном случае сеть начнет генерировать символы наугад.]

Из основного задания, попробуйте сгенерировать небольшой рассказик. Поиграйте с параметрами. Найдите собственный датасет( чтобы проверяющим было весело ) и попробуйте обучить нейронную сеть на нем. 

Не забывайте эксперементировать со структурой. У LSTM много аналогий, посмотрите как они справляются с данной задачей и запишите это в отчет. 

**Вопрос 7**

Введите результат модели. Помните, он должен быть осмысленным.


In [0]:
def sample(net, size, prime='The', top_k=None, cuda=False):
    net.eval()
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = None
    for ch in prime:
        char, h = net.predict(ch, h,top_k=top_k)
        
    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = net.predict(chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [0]:
net.eval()
print(sample(net, 25, prime='Путин', top_k=2, cuda=True))

Посмотрим на семлы случайной модели.

In [0]:
print(sample(CharRNN(chars, n_hidden=num_hidden_units, n_layers=2).to(device), 
             20, prime='Путин ', top_k=5, cuda=True))

Разница видна, значит мы все сделали правильно.

In [0]:
with open('./rnn.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

### Вопрос 8


Тут предлагается описать все проделанные эксперименты. Отчет будет весить 2 балла из 10.  Попробуйте сделать его насыщенным необходимой информацией для проверяющих, чтобы можно было сразу оценить сколько было проделано работы. Пример вопросов для отчета был дан выше. Можно добавить семплы модели на эпохах 1, 3, 9, 15, для лучшего понимания обучения.